In [1]:
"""Møller-Plesset perturbation theory to second order"""

__author__    = "Roberto Di Remigio"
__credit__    = ["Roberto Di Remigio", "Xin Li"]

__copyright__ = "(c) 2021, ENCSS and PDC"
__license__   = "MIT"
__date__      = "2021-04-13"

<figure>
  <IMG SRC="../img/ENCCS-PDC-logos.jpg" WIDTH=150 ALIGN="right">
</figure>

# Møller-Plesset perturbation theory to second order

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

In Hartree-Fock (HF) theory, we approximate the molecular electronic wavefunction as a single Slater determinant:
\begin{equation}
  | 0 \rangle = ...
\end{equation}
where $\phi_{p}$ are the *molecular orbitals* (MOs). The HF energy is:
\begin{equation}
E_{\mathrm{HF}} = 
\sum_{i=1}^{N}\langle \phi_{i} | -\frac{1}{2}\nabla^{2} + V_{\mathrm{Ne}} | \phi_{i} \rangle + 
\frac{1}{2} \sum_{ij} (\langle \phi_{i}\phi_{j}| \phi_{i}\phi_{j} \rangle - \langle \phi_{i}\phi_{j}| \phi_{j}\phi_{i} \rangle).
\end{equation}

### TODO
- AO basis expansion and energy functional
- Variational optimization and Fock equations
- High-level description of the algorithm

In [3]:
import veloxchem as vlx
import numpy as np

h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
"""

mol = vlx.Molecule.from_xyz(h2o_xyz)
print(mol.get_string())

basis = vlx.MolecularBasis.read(mol, "sto-3g")
print(basis.get_string(mol))

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 12.
Molecular Geometry (Angstroms)

  Atom         Coordinate X          Coordinate Y          Coordinate Z  

  O           0.000000000000        0.000000000000        0.000000000000
  H           0.000000000000        0.740848095288        0.582094932012
  H           0.000000000000       -0.740848095288        0.582094932012

Molecular Basis (Atomic Basis)

Basis: STO-3G                                         

  Atom Contracted GTOs          Primitive GTOs           

  H   (1S)                     (3S)                     
  O   (2S,1P)                  (6S,3P)                  

Contracted Basis Functions : 7                        
Primitive Basis Functions  : 21                       



In [4]:
ovldrv = vlx.OverlapIntegralsDriver()
ovl = ovldrv.compute(mol, basis)
S = ovl.to_numpy()

In [5]:
kindrv = vlx.KineticEnergyIntegralsDriver()
kin = kindrv.compute(mol, basis)
T = kin.to_numpy()

In [6]:
naidrv = vlx.NuclearPotentialIntegralsDriver()
nai = naidrv.compute(mol, basis)
V = -nai.to_numpy()

In [7]:
eridrv = vlx.ElectronRepulsionIntegralsDriver()
eri = eridrv.compute_in_mem(mol, basis)

### Matrix and tensor manipulations with NumPy

TODO
1. `np.einsum`
2. `np.linalg.eigh` and `np.linalg.norm`

## Orthogonalization of the AO basis

Find $\mathbf{X}$ such that:
\begin{equation}
\mathbf{X}^{t} \mathbf{S} \mathbf{X} = \mathbf{1}
\end{equation}

In the absence of linear dependencies in the AO basis, the overlap matrix is symmetric (and positive-definite) It is thus diagonalizable in a basis of its eigenvectors:
\begin{equation}
\mathbf{S} = \mathbf{U}\mathbf{\sigma}\mathbf{U}^{t}
\end{equation}
with $\mathbf{\sigma} = \mathrm{diag}(\sigma_{0}, \cdots, \sigma_{n})$ the diagonal matrix of eigenvalues and $\mathbf{U}$ its eigenvectors. Any function of the overlap matrix can be defined in terms of its eigenvalues and eigenvectors. This gives us two choices for the orthogonalization:

- **Symmetric (Löwdin)**, with $\mathbf{X} = \mathbf{U}\mathbf{\sigma}^{-\frac{1}{2}}\mathbf{U}^{t}$.
- **Canonical**, with $\mathbf{X} = \mathbf{U}\mathbf{\sigma}^{-\frac{1}{2}}$.

In both cases, we achieve a factorization of the overlap matrix as $\mathbf{S} = \left(\mathbf{X}^{t}\mathbf{X}\right)^{-1}$.

In [8]:
#help(np.linalg.eigh)
sigma, U = np.linalg.eigh(S)

X = np.einsum("Pk,k,Qk->PQ", U, 1.0/np.sqrt(sigma), U, optimize=True)

# check that the orthogonalizer is correct
test_S = np.linalg.inv(np.einsum("Pk,Qk->PQ", X, X, optimize=True))
np.testing.assert_allclose(test_S, S, atol=1e-10)

In [9]:
def AO_to_OAO(matrix, transformer):
    return np.einsum("Pk,kl,Ql->PQ", transformer, matrix, transformer, optimize=True)

## Initial guess

We use the very simple core guess: the initial MO coefficients are the eigenvectors of the core Hamiltonian: $h = T + V$. The algoritm goes as follows:
1. Transform the core Hamiltonian $h$ to OAO basis.
2. Diagonalize it.
3. Backtransform the coefficients to AO basis.
4. Form the density matrix using $D_{\mu\nu} = \sum_{i=1}^{N_{\mathrm{O}}} C_{\mu i} C_{\nu i}$, where $N_{\mathrm{O}}$ is the number of occupied orbitals.

In [58]:
N_O = mol.number_of_electrons() // 2

h = T + V
# transform to orthogonal AO basis
h_OAO = AO_to_OAO(h, X)
# diagonalize
_, C_OAO = np.linalg.eigh(h_OAO)
# backtransform coefficients
C = np.einsum("kP,kQ->PQ", X, C_OAO)
# form  density matrix from occupied slice of coefficients
D = np.einsum("Pi,Qi->PQ", C[:, :N_O], C[:, :N_O], optimize=True)

We can check that the density matrix so produced is idempotent:
\begin{equation}
\mathbf{D}\mathbf{S}\mathbf{D} = \mathbf{D}
\end{equation}
and normalized to the number of electrons:
\begin{equation}
N = 2\,\mathrm{tr} \, \mathbf{D}\mathbf{S} = 2 \sum_{\mu\nu}D_{\mu\nu}S_{\nu\mu}
\end{equation}

These checks will ensure that our implementation produces sane density matrices!

In [66]:
# check that the density matrix is normalized to the number of electrons
np.testing.assert_allclose(2.*np.einsum("ij,ji->", D, S), mol.number_of_electrons(), atol=1e-10)
# check that the density matrix is idempotent
np.testing.assert_allclose(np.einsum("Pk,kl,lQ->PQ", D, S, D), D, atol=1e-10)

The initial energy is:
\begin{equation}
E^{[0]} = E_{\mathrm{NN}} + \mathrm{tr}\,\mathbf{h}\mathbf{D}^{[0]}
\end{equation}

In [74]:
E_NN = mol.nuclear_repulsion_energy()
E = np.einsum("pq,qp->", h, D, optimize=True) + E_NN
print(f"Initial SCF energy: {E:20.12f}")

Initial SCF energy:     -51.967424602882


## The iterative procedure

With a guess density at hand, we can start the Roothaan-Hall self-consistent iterations. We will run until a convergence criterion is met, `conv_thresh`, or until we exceed the maximum number of iterations, `max_iter`.
We choose the *norm of the difference between two successive density matrices*, $\| \mathbf{D}^{[n+1]} - \mathbf{D}^{[n]} \|$,  as convergence criterion.

In [72]:
max_iter = 50
conv_thresh = 1e-6

At each iteration, we form the *Coulomb*, $J$, and *exchange*, $K$, matrices from a contraction of the density matrix and the pre-computed ERI tensor:
\begin{equation}
  J^{[n]}_{\mu\nu} = \sum_{\kappa\lambda}(\mu\nu|\kappa\lambda)D_{\lambda\kappa}^{[n]}
\end{equation}
and 
\begin{equation}
  K^{[n]}_{\mu\nu} = \sum_{\kappa\lambda}(\mu\kappa|\nu\lambda)D_{\lambda\kappa}^{[n]}
\end{equation}

Finally, the Fock matrix is:
\begin{equation}
\mathbf{F} = \mathbf{h} + 2\mathbf{J} - \mathbf{K}
\end{equation}

In [70]:
print("# it.    SCF energy (E_h)    |Delta_E|       Delta_D")
print(f"  0  {E:20.12f}        N/A            N/A")

for it in range(1, max_iter+1):
    # build Fock matrix
    J = np.einsum("PQrs,sr->PQ", eri, D, optimize=True)
    K = np.einsum("PrQs,sr->PQ", eri, D, optimize=True)
    F = h + 2*J - K
    
    # transform Fock matrix to orthogonal AO basis
    F_OAO = AO_to_OAO(F, X)
    
    # diagonalize
    _, C_OAO = np.linalg.eigh(F_OAO)
    
    # backtransform coefficients
    C = np.einsum("kP,kQ->PQ", X, C_OAO)
    
    # form density matrix from occupied slice of coefficients
    D_ = np.einsum("Pi,Qi->PQ", C[:, :N_O], C[:, :N_O], optimize=True)
    
    # evaluate energy
    E_ = np.einsum("pq,qp->", h+F, D_, optimize=True) + E_NN
    
    # compute convergence metric
    Delta_D = np.linalg.norm(D_ - D)
    
    # print iteration stats
    print(f" {it:>2d}  {E_:20.12f}   {np.abs(E_ - E):7.5e}    {Delta_D:7.5e}   {'CONVERGED!' if Delta_D < threshold else ''}")
    
    # update current density and current energy
    D = D_
    E = E_
    
    # check if convergence threshold was met
    if Delta_D < conv_thresh:
        print("Roothaan-Hall iterations converged!")
        break
    
    if it == max_iter:
        raise RuntimeError("Maximum number of Roothaan-Hall exceeded!")

# it.    SCF energy (E_h)    |Delta_E|       Delta_D
  0      -51.967424601282        N/A            N/A
  1      -74.960337070648   2.29929e+01    1.43751e-08   CONVERGED!
Roothaan-Hall iterations converged!


Let's compare the result of our own SCF with that obtainable from VeloxChem.

In [55]:
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(mol, basis)
ref_E = scfdrv.get_scf_energy()

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [56]:
print(f"Difference with reference result: {E-ref_E:5.8e}")

Difference with reference result: -1.11210667e-07


## Discussion points

- Convergence patterns
- Convergence acceleration
- Scaling